# Introduction

This is my Notebook on ESO competition to predict the Battery ouput  I will do the followoing:
* Perform **[EDA and Feature Engineering](#eda)** to clean the dataset, add new features to try to improve the accuracy of the prediction and understand  seasonality for Baterry Forecast
* Use **[XGBoost](#xgboost)** model to predict Battery output.

* Use **[Prophet](#prophet)** model to predict Battery output.
* Use **[LightGBM](#lgbm)**   to predict Battery output.
* Compare the results from the different models, draw key insights and provide final output for submission.

#  Table of contents
* [EDA and Feature Engineering](#eda_fe)
* [XGBoost](#xgboost)
* [Prophet](#prophet)
* [LightGBM](#lgbm)

In [49]:
import pandas as pd

In [50]:
df = pd.read_csv("test_data.csv", index_col= 0)

In [51]:
df

,UTC_Settlement_DateTime,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,...,is_daySouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
id,,,,,,,,,,,,,,,,,,,,,
39457,11/11/2022 00:00,-3.696,-3.8850,-3.696,-4.0635,-3.8850,-4.4310,-4.2525,-3.8850,-3.8850,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
39458,11/11/2022 00:30,-0.210,-0.2205,-0.210,-0.2310,-0.2205,-0.2520,-0.2415,-0.2205,-0.2205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39459,11/11/2022 01:00,0.924,0.9660,0.924,1.0185,0.9660,1.1130,1.0605,0.9660,0.9660,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
39460,11/11/2022 01:30,-4.200,-4.4100,-4.200,-4.6200,-4.4100,-5.0400,-4.8300,-4.4100,-4.4100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39461,11/11/2022 02:00,-0.672,-0.7035,-0.672,-0.7350,-0.7035,-0.8085,-0.7770,-0.7035,-0.7035,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49340,05/06/2023 21:30,14.847,15.5925,14.847,16.3275,15.5925,17.8185,17.0730,15.5925,15.5925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49341,05/06/2023 22:00,15.120,15.8760,15.120,16.6320,15.8760,18.1440,17.3880,15.8760,15.8760,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
49342,05/06/2023 22:30,14.700,15.4350,14.700,16.1700,15.4350,17.6400,16.9050,15.4350,15.4350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
column_names = df.columns.tolist()
print(column_names)

['UTC_Settlement_DateTime', 'East_Midlands_price', 'Eastern_England_price', 'London_price', 'Merseyside_and_Northern_Wales_price', 'North_Western_England_price', 'Northern_Scotland_price', 'South_Western_England_price', 'Southern_England_price', 'Southern_Scotland_price', 'West_Midlands_price', 'Yorkshire_price', 'temperature_2mBelfast_weather', 'relativehumidity_2mBelfast_weather', 'dewpoint_2mBelfast_weather', 'apparent_temperatureBelfast_weather', 'precipitationBelfast_weather', 'rainBelfast_weather', 'snowfallBelfast_weather', 'weathercodeBelfast_weather', 'surface_pressureBelfast_weather', 'cloudcoverBelfast_weather', 'cloudcover_lowBelfast_weather', 'cloudcover_midBelfast_weather', 'cloudcover_highBelfast_weather', 'windspeed_10mBelfast_weather', 'windspeed_100mBelfast_weather', 'winddirection_10mBelfast_weather', 'winddirection_100mBelfast_weather', 'windgusts_10mBelfast_weather', 'is_dayBelfast_weather', 'locationBelfast_weather', 'populationBelfast_weather', 'latitudeBelfast_w

In [53]:
print('Column_names: ')
for name in column_names:
    print(f'{name}')

Column_names: 
UTC_Settlement_DateTime
East_Midlands_price
Eastern_England_price
London_price
Merseyside_and_Northern_Wales_price
North_Western_England_price
Northern_Scotland_price
South_Western_England_price
Southern_England_price
Southern_Scotland_price
West_Midlands_price
Yorkshire_price
temperature_2mBelfast_weather
relativehumidity_2mBelfast_weather
dewpoint_2mBelfast_weather
apparent_temperatureBelfast_weather
precipitationBelfast_weather
rainBelfast_weather
snowfallBelfast_weather
weathercodeBelfast_weather
surface_pressureBelfast_weather
cloudcoverBelfast_weather
cloudcover_lowBelfast_weather
cloudcover_midBelfast_weather
cloudcover_highBelfast_weather
windspeed_10mBelfast_weather
windspeed_100mBelfast_weather
winddirection_10mBelfast_weather
winddirection_100mBelfast_weather
windgusts_10mBelfast_weather
is_dayBelfast_weather
locationBelfast_weather
populationBelfast_weather
latitudeBelfast_weather
longitudeBelfast_weather
generationtime_msBelfast_weather
utc_offset_secondsBel

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9888 entries, 39457 to 49344
Columns: 292 entries, UTC_Settlement_DateTime to elevationSouthhampton - Portsmouth_weather
dtypes: float64(261), object(31)
memory usage: 22.1+ MB


In [55]:
df.select_dtypes(include=['object'])

,UTC_Settlement_DateTime,locationBelfast_weather,timezoneBelfast_weather,timezone_abbreviationBelfast_weather,locationBirmingham_weather,timezoneBirmingham_weather,timezone_abbreviationBirmingham_weather,locationGlasgow_weather,timezoneGlasgow_weather,timezone_abbreviationGlasgow_weather,...,timezone_abbreviationManchester_weather,locationNewcastle upon Tyne_weather,timezoneNewcastle upon Tyne_weather,timezone_abbreviationNewcastle upon Tyne_weather,locationSheffield_weather,timezoneSheffield_weather,timezone_abbreviationSheffield_weather,locationSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather
id,,,,,,,,,,,,,,,,,,,,,
39457,11/11/2022 00:00,Belfast,GMT,GMT,Birmingham,GMT,GMT,Glasgow,GMT,GMT,...,GMT,Newcastle upon Tyne,GMT,GMT,Sheffield,GMT,GMT,Southhampton - Portsmouth,GMT,GMT
39458,11/11/2022 00:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39459,11/11/2022 01:00,Belfast,GMT,GMT,Birmingham,GMT,GMT,Glasgow,GMT,GMT,...,GMT,Newcastle upon Tyne,GMT,GMT,Sheffield,GMT,GMT,Southhampton - Portsmouth,GMT,GMT
39460,11/11/2022 01:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39461,11/11/2022 02:00,Belfast,GMT,GMT,Birmingham,GMT,GMT,Glasgow,GMT,GMT,...,GMT,Newcastle upon Tyne,GMT,GMT,Sheffield,GMT,GMT,Southhampton - Portsmouth,GMT,GMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49340,05/06/2023 21:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49341,05/06/2023 22:00,Belfast,GMT,GMT,Birmingham,GMT,GMT,Glasgow,GMT,GMT,...,GMT,Newcastle upon Tyne,GMT,GMT,Sheffield,GMT,GMT,Southhampton - Portsmouth,GMT,GMT
49342,05/06/2023 22:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So we have some 31 columns that are object and need to be translated into numbers with some kind of encoding. Also we need to sort out the time series element of the dataframe

In [56]:
df.isna().any().head(20)

UTC_Settlement_DateTime                False
East_Midlands_price                    False
Eastern_England_price                  False
London_price                           False
Merseyside_and_Northern_Wales_price    False
North_Western_England_price            False
Northern_Scotland_price                False
South_Western_England_price            False
Southern_England_price                 False
Southern_Scotland_price                False
West_Midlands_price                    False
Yorkshire_price                        False
temperature_2mBelfast_weather           True
relativehumidity_2mBelfast_weather      True
dewpoint_2mBelfast_weather              True
apparent_temperatureBelfast_weather     True
precipitationBelfast_weather            True
rainBelfast_weather                     True
snowfallBelfast_weather                 True
weathercodeBelfast_weather              True
dtype: bool

In [57]:
df.isna().any().head(20)

UTC_Settlement_DateTime                False
East_Midlands_price                    False
Eastern_England_price                  False
London_price                           False
Merseyside_and_Northern_Wales_price    False
North_Western_England_price            False
Northern_Scotland_price                False
South_Western_England_price            False
Southern_England_price                 False
Southern_Scotland_price                False
West_Midlands_price                    False
Yorkshire_price                        False
temperature_2mBelfast_weather           True
relativehumidity_2mBelfast_weather      True
dewpoint_2mBelfast_weather              True
apparent_temperatureBelfast_weather     True
precipitationBelfast_weather            True
rainBelfast_weather                     True
snowfallBelfast_weather                 True
weathercodeBelfast_weather              True
dtype: bool

In [58]:
df['UTC_Settlement_DateTime'] = pd.to_datetime(df['UTC_Settlement_DateTime'], format= 'mixed')


In [59]:
df.set_index("UTC_Settlement_DateTime", inplace=True)
df.sort_index(inplace=True)


In [60]:
df

,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,West_Midlands_price,...,is_daySouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
UTC_Settlement_DateTime,,,,,,,,,,,,,,,,,,,,,
2022-01-12 00:00:00,35.071785,37.717785,35.797230,40.363785,37.717785,45.655785,43.009785,37.717785,37.717785,37.717785,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-01-12 00:30:00,38.851785,41.686785,38.851785,44.521785,41.686785,50.191785,47.356785,41.686785,41.686785,41.686785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 01:00:00,35.533785,38.200785,35.797230,40.867785,38.200785,46.212285,43.545285,38.200785,38.200785,38.200785,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-01-12 01:30:00,33.874785,36.457785,35.797230,39.051285,36.457785,44.217285,41.634285,36.457785,36.457785,36.457785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 02:00:00,33.018090,35.056875,35.797230,36.940785,34.452285,41.928285,39.439785,34.452285,34.452285,34.452285,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-05 21:30:00,17.010000,17.860500,17.010000,18.711000,17.860500,20.412000,19.561500,17.860500,17.860500,17.860500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-05 22:00:00,19.918500,20.916000,19.918500,21.913500,20.916000,23.898000,22.911000,20.916000,20.916000,20.916000,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2023-12-05 22:30:00,17.892000,18.784500,17.892000,19.677000,18.784500,21.472500,20.580000,18.784500,18.784500,18.784500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df = df[df.columns.drop(list(df.filter(regex='is_')))]


We have used this code to get rid of all the columns that start with is_ as it adds nothing in principle, we might do something along those lines later on

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9888 entries, 2022-01-12 00:00:00 to 2023-12-05 23:30:00
Columns: 281 entries, East_Midlands_price to elevationSouthhampton - Portsmouth_weather
dtypes: float64(251), object(30)
memory usage: 21.3+ MB


In [63]:
df.head(50)

,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,West_Midlands_price,...,windgusts_10mSouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
UTC_Settlement_DateTime,,,,,,,,,,,,,,,,,,,,,
2022-01-12 00:00:00,35.071785,37.717785,35.797230,40.363785,37.717785,45.655785,43.009785,37.717785,37.717785,37.717785,...,22.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-01-12 00:30:00,38.851785,41.686785,38.851785,44.521785,41.686785,50.191785,47.356785,41.686785,41.686785,41.686785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 01:00:00,35.533785,38.200785,35.797230,40.867785,38.200785,46.212285,43.545285,38.200785,38.200785,38.200785,...,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-01-12 01:30:00,33.874785,36.457785,35.797230,39.051285,36.457785,44.217285,41.634285,36.457785,36.457785,36.457785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 02:00:00,33.018090,35.056875,35.797230,36.940785,34.452285,41.928285,39.439785,34.452285,34.452285,34.452285,...,21.6,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-01-12 02:30:00,33.018090,35.056875,35.797230,36.625785,34.147785,41.571285,39.093285,34.226010,34.147785,34.147785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 03:00:00,33.018090,35.056875,35.797230,37.518285,35.008785,42.558285,40.038285,35.008785,35.008785,35.008785,...,21.6,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2022-01-12 03:30:00,33.018090,35.056875,35.797230,35.982870,33.476100,40.111785,37.696785,34.226010,33.762960,33.726735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 04:00:00,35.470785,38.137785,35.797230,40.804785,38.137785,46.138785,43.471785,38.137785,38.137785,38.137785,...,21.6,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0


In [64]:
for col in df.columns:
    most_common_value = df[col].mode()[0]
    df[col].fillna(most_common_value, inplace=True)

C:\Users\ivan.sanz\AppData\Local\Temp\ipykernel_13280\955556545.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna(most_common_value, inplace=True)


In [65]:
df.tail(7)

,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,West_Midlands_price,...,windgusts_10mSouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
UTC_Settlement_DateTime,,,,,,,,,,,,,,,,,,,,,
2023-12-05 20:30:00,19.2150,20.1810,19.2150,21.1365,20.1810,23.0580,22.0920,20.1810,20.1810,20.1810,...,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2023-12-05 21:00:00,19.3830,20.3490,19.3830,21.3255,20.3490,23.2575,22.2915,20.3490,20.3490,20.3490,...,53.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2023-12-05 21:30:00,17.0100,17.8605,17.0100,18.7110,17.8605,20.4120,19.5615,17.8605,17.8605,17.8605,...,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2023-12-05 22:00:00,19.9185,20.9160,19.9185,21.9135,20.9160,23.8980,22.9110,20.9160,20.9160,20.9160,...,50.8,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2023-12-05 22:30:00,17.8920,18.7845,17.8920,19.6770,18.7845,21.4725,20.5800,18.7845,18.7845,18.7845,...,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2023-12-05 23:00:00,19.9500,20.9475,19.9500,21.9450,20.9475,23.9400,22.9425,20.9475,20.9475,20.9475,...,49.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2023-12-05 23:30:00,17.3250,18.1860,17.3250,19.0575,18.1860,20.7900,19.9290,18.1860,18.1860,18.1860,...,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0


Ok, this last line of code was not be best but it has worked so now lets do the one hot encoding so we have all numbers

In [66]:
df = df[df.columns.drop(list(df.filter(regex='timezone')))]

In [67]:
df

,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,Southern_England_price,Southern_Scotland_price,West_Midlands_price,...,winddirection_10mSouthhampton - Portsmouth_weather,winddirection_100mSouthhampton - Portsmouth_weather,windgusts_10mSouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
UTC_Settlement_DateTime,,,,,,,,,,,,,,,,,,,,,
2022-01-12 00:00:00,35.071785,37.717785,35.797230,40.363785,37.717785,45.655785,43.009785,37.717785,37.717785,37.717785,...,54.0,49.0,22.7,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2022-01-12 00:30:00,38.851785,41.686785,38.851785,44.521785,41.686785,50.191785,47.356785,41.686785,41.686785,41.686785,...,54.0,252.0,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2022-01-12 01:00:00,35.533785,38.200785,35.797230,40.867785,38.200785,46.212285,43.545285,38.200785,38.200785,38.200785,...,49.0,43.0,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2022-01-12 01:30:00,33.874785,36.457785,35.797230,39.051285,36.457785,44.217285,41.634285,36.457785,36.457785,36.457785,...,54.0,252.0,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2022-01-12 02:00:00,33.018090,35.056875,35.797230,36.940785,34.452285,41.928285,39.439785,34.452285,34.452285,34.452285,...,51.0,44.0,21.6,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-05 21:30:00,17.010000,17.860500,17.010000,18.711000,17.860500,20.412000,19.561500,17.860500,17.860500,17.860500,...,54.0,252.0,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2023-12-05 22:00:00,19.918500,20.916000,19.918500,21.913500,20.916000,23.898000,22.911000,20.916000,20.916000,20.916000,...,21.0,23.0,50.8,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0
2023-12-05 22:30:00,17.892000,18.784500,17.892000,19.677000,18.784500,21.472500,20.580000,18.784500,18.784500,18.784500,...,54.0,252.0,22.3,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,10.0


In [68]:
 

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Example list of strings
data = ['Belfast', 'Birmingham', 'Glasgow', 'Leeds', 'Liverpool', 'London', 'Manchester', 'Newcastle upon Tyne', 'Sheffield' , 'Southhampton - Portsmouth' ]
# Convert the list of strings into numerical values
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data)
# Perform one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# Create dictionary with original values as keys and one-hot encoded values as values
hot_encoded_dict = {k:v for k,v in zip(data, onehot_encoded)}
hot_encoded_dict

c:\Users\ivan.sanz\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


{'Belfast': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Birmingham': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Glasgow': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 'Leeds': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 'Liverpool': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 'London': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'Manchester': array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 'Newcastle upon Tyne': array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 'Sheffield': array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 'Southhampton - Portsmouth': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])}

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9888 entries, 2022-01-12 00:00:00 to 2023-12-05 23:30:00
Columns: 261 entries, East_Midlands_price to elevationSouthhampton - Portsmouth_weather
dtypes: float64(251), object(10)
memory usage: 19.8+ MB


Ok after some consideration I am going to drop all the object columns as I don't think they are that relevant. The data already maps to the location so yeah.

In [70]:
df = df.select_dtypes(exclude=['object'])

In [71]:
df.to_csv('Test_sorted.csv', index=True)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9888 entries, 2022-01-12 00:00:00 to 2023-12-05 23:30:00
Columns: 251 entries, East_Midlands_price to elevationSouthhampton - Portsmouth_weather
dtypes: float64(251)
memory usage: 19.0 MB
